https://codingandfun.com/backtest-mean-reversion-strategy-python/

In [2]:
import requests 
import pandas as pd 
import numpy as np 

In [3]:
from cpl.data.processor_binance import BinanceProcessor

In [5]:
ticker_list = ["BTCUSDT"]
start_date = "2021-12-15"
end_date = "2021-12-31"
time_interval = "1m"
tech_inds = ['rsi','macd']

In [6]:
p = BinanceProcessor("binance")
df = p.download_data(ticker_list, start_date, end_date, time_interval)
df.head()

,time,open,high,low,close,volume,adj_close,tic
0,2021-12-15 00:00:00,48120.88,48157.41,48120.87,48136.00,14.51234,48136.00,BTCUSDT
1,2021-12-15 00:01:00,48136.00,48145.09,48112.28,48129.20,9.31538,48129.20,BTCUSDT
2,2021-12-15 00:02:00,48129.20,48185.21,48126.67,48184.64,12.80545,48184.64,BTCUSDT
3,2021-12-15 00:03:00,48184.64,48186.88,48173.02,48178.98,6.95185,48178.98,BTCUSDT
4,2021-12-15 00:04:00,48178.99,48189.66,48156.08,48187.29,17.31493,48187.29,BTCUSDT


In [7]:
stockprices = df.set_index('time')

In [9]:
stockprices = stockprices.iloc[::-1]

stockprices['20d'] = stockprices['close'].rolling(20).mean() 


stockprices['return'] = np.log(stockprices['close'] / stockprices['close'].shift(1) )


stockprices['difference'] = stockprices['close'] - stockprices['20d']

stockprices['long'] = np.where(stockprices['difference'] < -2 ,1,np.nan)
stockprices['long'] = np.where(stockprices['difference'] * stockprices['difference'].shift(1) < 0, 0, stockprices['long'])
stockprices['long'] = stockprices['long'].ffill().fillna(0)

stockprices['gain_loss'] = stockprices['long'].shift(1) * stockprices['return']
stockprices = stockprices.dropna(subset=['20d'])

stockprices['total'] =  stockprices['gain_loss'].cumsum()
print(stockprices.tail(30))

                         open      high       low     close    volume  \
time                                                                    
2021-12-15 00:29:00  48250.00  48263.40  48244.24  48260.02  10.61030   
2021-12-15 00:28:00  48254.98  48254.98  48220.01  48250.00  17.53893   
2021-12-15 00:27:00  48275.96  48275.97  48250.00  48254.97  10.61372   
2021-12-15 00:26:00  48296.65  48296.65  48264.56  48275.96  12.23090   
2021-12-15 00:25:00  48279.23  48298.90  48278.60  48296.65  10.84330   
2021-12-15 00:24:00  48247.36  48322.04  48233.60  48279.23  39.13680   
2021-12-15 00:23:00  48236.60  48265.00  48236.59  48247.35  14.76061   
2021-12-15 00:22:00  48219.66  48250.73  48219.66  48236.60  13.89313   
2021-12-15 00:21:00  48232.64  48247.79  48217.92  48219.66  21.37585   
2021-12-15 00:20:00  48205.84  48262.96  48201.81  48232.64  31.57672   
2021-12-15 00:19:00  48172.55  48240.18  48172.54  48205.84  28.17239   
2021-12-15 00:18:00  48150.58  48198.00  48150.57  